# <center> Sweeping variables in tproc v2 demonstration

<center> In this demo you will sweep the amplitude of a pulse in loopback to demonstrate control over the QICK. 


Imports

In [ ]:
# jupyter setup boilerplate
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from qick import *

# for now, all the tProc v2 classes need to be individually imported (can't use qick.*)

# the main program class
from qick.asm_v2 import AveragerProgramV2
# for defining sweeps
from qick.asm_v2 import QickSpan, QickSweep1D

Connect to RFSoC using Pyro4

In [ ]:
import Pyro4
from qick import QickConfig
Pyro4.config.SERIALIZER = "pickle"
Pyro4.config.PICKLE_PROTOCOL_VERSION=4

ns_host = "192.168.1.135"
ns_port = 8000
proxy_name = "rfsoc"

ns = Pyro4.locateNS(host=ns_host, port=ns_port)
soc = Pyro4.Proxy(ns.lookup(proxy_name))
soccfg = QickConfig(soc.get_cfg())
print(soccfg)

### Hardware Configuration

In [ ]:
# DAC Signal Generating Channels
GEN_CH0 = 0
GEN_CH1 = 1
GEN_CH2 = 2
GEN_CH3 = 3
GEN_CH4 = 4
GEN_CH5 = 5
GEN_CH6 = 6
GEN_CH7 = 7
GEN_CH8 = 8
GEN_CH9 = 9
GEN_CH10 = 10
GEN_CH11 = 11

# ADC Readout Channels
RO_CH0 = 0
RO_CH1 = 1
RO_CH2 = 2
RO_CH3 = 3
RO_CH4 = 4
RO_CH5 = 5

### Basic Sweep Programs (1 Dimensional)

First, we will sweep over the amplitdue (gain) of the pulse

In [ ]:
class SimpleSweepProgram(AveragerProgramV2):
    def _initialize(self, cfg):
        ro_ch = cfg['ro_ch']
        gen_ch = cfg['gen_ch']
        
        self.declare_gen(ch=gen_ch, nqz=1)
        self.declare_readout(ch=ro_ch, length=cfg['ro_len'], freq=cfg['freq'], gen_ch=gen_ch)

        self.add_loop("myloop", self.cfg["steps"])

        self.add_pulse(ch=gen_ch, name="myconst", ro_ch=ro_ch, 
                       style="const", 
                       length=cfg['pulse_len'], 
                       freq=cfg['freq'], 
                       phase=cfg['pulse_phase'],
                       gain=cfg['pulse_gain'],
                      )
        
    def _body(self, cfg):
        self.pulse(ch=cfg['gen_ch'], name="myconst", t=0)
        self.trigger(ros=[cfg['ro_ch']], pins=[0], t=cfg['trig_time'])


In [ ]:
# do a sweep with 5 points and plot decimated
config = {
    ## Sweep Params: ##
    'steps': 5,
    ## Channel Params. ##
    'gen_ch': GEN_CH0,
    'ro_ch': RO_CH0,
    ## Pulse Params. ##
    'freq': 1000, # [MHz]
    'pulse_len': 0.1, # [us]
    'pulse_phase': 0, # [deg]
    'pulse_gain': QickSweep1D("myloop", 0.0, 1.0), # [DAC units]
    ## Readout Params. ##
    'trig_time': 0.35, # [us]
    'ro_len': 0.3, # [us]
     }

prog = SimpleSweepProgram(soccfg, reps=1, final_delay=0.5, cfg=config)

iq_list = prog.acquire_decimated(soc, soft_avgs=100)
t = prog.get_time_axis(ro_index=0)

In [ ]:
for ii, iq in enumerate(iq_list[0]):
    # plt.plot(t, iq[:,0], label="I value, step %d"%(ii))
    # plt.plot(iq[:,1], label="Q value, step %d"%(ii))
    plt.plot(np.abs(iq.dot([1,1j])), label="mag, step %d"%(ii))
plt.legend()
plt.ylabel("a.u.")
plt.xlabel("us");

### Sweeping over multiple variables (still 1D)

Now we will sweep over both the pulse phase and gain

In [ ]:
class SimpleSweepProgram(AveragerProgramV2):
    def _initialize(self, cfg):
        ro_ch = cfg['ro_ch']
        gen_ch = cfg['gen_ch']
        
        self.declare_gen(ch=gen_ch, nqz=1)
        self.declare_readout(ch=ro_ch, length=cfg['ro_len'], freq=cfg['freq'], gen_ch=gen_ch)

        self.add_loop("myloop", self.cfg["steps"])

        self.add_pulse(ch=gen_ch, name="myconst", ro_ch=ro_ch, 
                       style="const", 
                       length=cfg['pulse_len'], 
                       freq=cfg['freq'], 
                       phase=cfg['pulse_phase'],
                       gain=cfg['pulse_gain'],
                      )
        
    def _body(self, cfg):
        self.pulse(ch=cfg['gen_ch'], name="myconst", t=0)
        self.trigger(ros=[cfg['ro_ch']], pins=[0], t=cfg['trig_time'])


In [ ]:
# do a sweep with 5 points and plot decimated
config = {
    ## Sweep Params: ##
    'steps': 5,
    ## Channel Params. ##
    'gen_ch': GEN_CH0,
    'ro_ch': RO_CH0,
    ## Pulse Params. ##
    'freq': 1000, # [MHz]
    'pulse_len': 0.1, # [us]
    'pulse_phase': QickSweep1D("myloop", -360, 720), # [deg]
    'pulse_gain': QickSweep1D("myloop", 0.0, 1.0), # [DAC units]
    ## Readout Params. ##
    'trig_time': 0.35, # [us]
    'ro_len': 0.3, # [us]
     }

prog = SimpleSweepProgram(soccfg, reps=1, final_delay=0.5, cfg=config)

iq_list = prog.acquire_decimated(soc, soft_avgs=100)
t = prog.get_time_axis(ro_index=0)

In [ ]:
for ii, iq in enumerate(iq_list[0]):
    plt.plot(t, iq[:,0], label="I value, step %d"%(ii))
    # plt.plot(iq[:,1], label="Q value, step %d"%(ii))
    # plt.plot(np.abs(iq.dot([1,1j])), label="mag, step %d"%(ii))
plt.legend()
plt.ylabel("a.u.")
plt.xlabel("us");

We can also do higher-resolution sweeps using the <code> acquire()</code> function

In [ ]:
config['steps']=201
prog = SimpleSweepProgram(soccfg, reps=100, final_delay=1.0, cfg=config)
iq_list = prog.acquire(soc, soft_avgs=1, progress=True)
# plt.plot(np.angle(iq_list[0][0].dot([1,1j]), deg=True))
plt.plot(iq_list[0][0,:,0], iq_list[0][0,:,1])
plt.ylabel("Q")
plt.xlabel("I");

Lastly, we can also sweep the length of the pulses

In [ ]:
# do a sweep with 5 points and plot decimated
config = {
    ## Sweep Params: ##
    'steps': 5,
    ## Channel Params. ##
    'gen_ch': GEN_CH0,
    'ro_ch': RO_CH0,
    ## Pulse Params. ##
    'freq': 1000, # [MHz]
    'pulse_len': QickSweep1D('myloop', 0.05, 0.15), # [us]
    'pulse_phase': 0, # [deg]
    'pulse_gain': 1.0, # [DAC units]
    ## Readout Params. ##
    'trig_time': 0.35, # [us]
    'ro_len': 0.3, # [us]
     }

prog = SimpleSweepProgram(soccfg, reps=1, final_delay=0.5, cfg=config)

iq_list = prog.acquire_decimated(soc, soft_avgs=100)
t = prog.get_time_axis(ro_index=0)

In [ ]:
for ii, iq in enumerate(iq_list[0]):
    # plt.plot(t, iq[:,0], label="I value, step %d"%(ii))
    # plt.plot(iq[:,1], label="Q value, step %d"%(ii))
    plt.plot(np.abs(iq.dot([1,1j])), label="mag, step %d"%(ii))
plt.legend()
plt.ylabel("a.u.")
plt.xlabel("us");

### Multi-Dimensional Loops and Sweeps

If you want an N-dimensional looping program, you just need to call add_loop() multiple times. Each swept value should reference the name of the loop you want the sweep to happen in. (You can sweep over the "reps" loop if you want, but then your averaging will not make sense.)

If you want a value that gets swept in more than one loop level, you need to build a sweep object by summing the initial value with some "spans," where each span is the range that you sweep over in a given loop.
QickSweep1D is actually just a convenience function that builds a 1-D sweep using this machinery.

Just for fun, we'll also trigger the DDR4 and MR buffers in this program.

In [ ]:
class Sweep2DProgram(AveragerProgramV2):
    def _initialize(self, cfg):
        ro_ch = cfg['ro_ch']
        gen_ch = cfg['gen_ch']
        
        self.declare_gen(ch=gen_ch, nqz=1)
        self.declare_readout(ch=ro_ch, length=cfg['ro_len'], freq=cfg['freq'], gen_ch=gen_ch)

        self.add_loop("loop1", self.cfg["steps1"]) # outer loop
        self.add_loop("loop2", self.cfg["steps2"]) # inner loop
        # the reps loop is always outermost
        # so the order and the shape of the raw data will be (reps, loop1, loop2)

        self.add_pulse(ch=gen_ch, name="myconst", ro_ch=ro_ch, 
                       style="const", 
                       length=cfg['pulse_len'], 
                       freq=cfg['freq'], 
                       phase=cfg['pulse_phase'],
                       gain=cfg['pulse_gain'],
                      )
        
    def _body(self, cfg):
        self.pulse(ch=cfg['gen_ch'], name="myconst", t=0)
        self.trigger(ros=[cfg['ro_ch']], pins=[0], t=cfg['trig_time'])


In [ ]:
# We will sweep over the gain in both loop levels
config = {
    ## Sweep Params: ##
    'steps1': 100,
    'steps2': 50,
    ## Channel Params. ##
    'gen_ch': GEN_CH0,
    'ro_ch': RO_CH0,
    ## Pulse Params. ##
    'freq': 1000, # [MHz]
    'pulse_len': 0.1, # [us]
    'pulse_phase': 0, # [deg]
    # initial value 0.4, sweep by 0.5 in loop1 and by 0.1 in loop2, in other words:
    # the very first shot is 0.4
    # the first loop2 sweep runs from 0.4 to 0.5
    # the last loop2 sweep runs from 0.9 to 1.0
    'pulse_gain': 0.4 + QickSpan("loop1", 0.5) + QickSpan("loop2", 0.1), # [DAC units]
    ## Readout Params. ##
    'trig_time': 0.35, # [us]
    'ro_len': 0.3, # [us]
     }

prog = Sweep2DProgram(soccfg, reps=100, final_delay=0.5, cfg=config)

iq_list = prog.acquire(soc, soft_avgs=100, progress=True)
t = prog.get_time_axis(ro_index=0)

In [ ]:
mag = np.abs(iq_list[0][0].dot([1,1j]))
plt.colorbar(plt.pcolormesh(mag/mag.max()))
plt.title("magnitude")
plt.ylabel("step index, loop 1")
plt.xlabel("step index, loop 2");

Now sweeping over both gain and phase

In [ ]:
# We will sweep over the gain in both loop levels
config = {
    ## Sweep Params: ##
    'steps1': 100,
    'steps2': 50,
    ## Channel Params. ##
    'gen_ch': GEN_CH0,
    'ro_ch': RO_CH0,
    ## Pulse Params. ##
    'freq': 1000, # [MHz]
    'pulse_len': 0.1, # [us]
    'pulse_phase': QickSweep1D('loop2', 0, 360), # [deg]
    'pulse_gain': QickSweep1D('loop1', 0.0, 1.0), # [DAC units]
    ## Readout Params. ##
    'trig_time': 0.35, # [us]
    'ro_len': 0.3, # [us]
     }

prog = Sweep2DProgram(soccfg, reps=100, final_delay=0.5, cfg=config)

iq_list = prog.acquire(soc, soft_avgs=10, progress=True)
t = prog.get_time_axis(ro_index=0)

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12,12))

# get the exact values of the parameters, after rounding - this also works for scalars, you will just get a float
phases = prog.get_pulse_param("myconst", "phase", as_array=True)
gains = prog.get_pulse_param("myconst", "gain", as_array=True)

plot = axes[0,0]
plt.colorbar(plot.pcolormesh(phases, gains, iq_list[0][0,:,:,0]), ax=plot)
plot.set_title("I")
plot.set_ylabel("gain")
plot.set_xlabel("phase")
plot = axes[0,1]
plt.colorbar(plot.pcolormesh(phases, gains, iq_list[0][0,:,:,1]), ax=plot)
plot.set_title("Q")
plot.set_ylabel("gain")
plot.set_xlabel("phase")

plot = axes[1,0]
plt.colorbar(plot.pcolormesh(phases, gains, np.abs(iq_list[0][0].dot([1,1j]))), ax=plot)
plot.set_title("magnitude")
plot.set_ylabel("gain")
plot.set_xlabel("phase")

plot = axes[1,1]
plt.colorbar(plot.pcolormesh(phases, gains, np.unwrap(np.angle(iq_list[0][0].dot([1,1j])), axis=1)), ax=plot)
plot.set_title("phase")
plot.set_ylabel("gain")
plot.set_xlabel("phase");

### Timeline Management
* you can sweep almost any time/duration parameter:
    * time parameter for pulse/trigger
    * time parameter for delay
    * length of a const pulse or the flat segment of a flat-top pulse
* "auto" (for pulse times and delay/wait) is smarter now
    * if your sweeps result in the end of the last pulse/readout getting swept relative to the reference time, "auto" should correctly account for this
    * if you use a delay that doesn't push the reference time past the end-of-pulse, the end-of-pulse will be decremented by the delay (in v1 this was all-or-nothing, sync_all would zero all end-of-pulse timestamps but synci would do nothing)
    * you can tell wait_auto/delay_auto to ignore generator times or readout times - the default (same as v1) is that wait_auto pays attention to readouts only, while delay_auto pays attention to both gens and ROs
* while "auto" is smarter now, I still want to encourage using explicit times when possible
    * this is easier now that all times and durations are in the same units, right?
    * my instinct is that it's safer if people "keep their hands on the wheel" - make them think about what they are doing, vs. letting them rely on safe-ish defaults
    * "auto" is still somewhat fragile - it doesn't know about jumps, there are probably some edge cases (multiple channels with swept times?) where the math doesn't work out
    * swept times consume registers, which are somewhat precious (print(prog) will list all the allocated registers)
    * for this reason, the default t in `pulse()` is now 0, not auto (as it was in v1)